# importing libraries needed. then use import.os to access kaggle API

In [1]:
import os
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine

os.environ['KAGGLE_USERNAME'] = "vinothnk"
os.environ['KAGGLE_KEY'] = "418378f21fb977c1f1ff44a5ebc60323"

In [2]:
#increase row and column size for better viewing of data
pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 25)

# download datasets from url using kaggle API

In [3]:
# import kaggle

# kaggle.api.dataset_download_files('olistbr/brazilian-ecommerce', path='./data', unzip=True)

In [4]:
#importing csv files into dataframes

customers_dataset = pd.read_csv('data/olist_customers_dataset.csv')
geolocation_dataset = pd.read_csv('data/olist_geolocation_dataset.csv')
order_items_dataset = pd.read_csv('data/olist_order_items_dataset.csv')
order_payments_dataset = pd.read_csv('data/olist_order_payments_dataset.csv')
order_reviews_dataset = pd.read_csv('data/olist_order_reviews_dataset.csv')
orders_dataset = pd.read_csv('data/olist_orders_dataset.csv')
products_dataset = pd.read_csv('data/olist_products_dataset.csv')
sellers_dataset = pd.read_csv('data/olist_sellers_dataset.csv')
product_category_name_translation_dataset = pd.read_csv('data/product_category_name_translation.csv')

# Functions to check the following:

#### 1. check the type of data types for the columns
#### 2. check the shape of the dataset (rows, columns)
#### 3. check for unique rows - if it matches the shape, no duplicates in the dataframe
#### 4. check for null values in dataframe

In [5]:
def check_data_types(dataframe):
    print(f"{key} has the following data types:")
    print(dataframe.dtypes)
    return

In [6]:
def check_data_shape(dataframe):
    print(f"The {key} has the following shape, {dataframe.shape[0]} rows & {dataframe.shape[1]} columns.")
    return

In [7]:
def check_unique_rows(dataframe):
    print(f"The {key} has {dataframe.drop_duplicates().shape[0]} unique rows.")
    return

In [8]:
def check_null_values(dataframe):
    print(f"The {key} has the following missing values:")
    print(dataframe.isnull().sum())
    return

# create a dictionary to call the name of each dataframe

In [9]:
# Create a dictionary with names as keys and dataframes as values
dataframes = {
    'customers_dataset': customers_dataset,
    'geolocation_dataset': geolocation_dataset,
    'order_items_dataset': order_items_dataset,
    'order_payments_dataset': order_payments_dataset,
    'order_reviews_dataset': order_reviews_dataset,
    'orders_dataset': orders_dataset,
    'products_dataset': products_dataset,
    'sellers_dataset': sellers_dataset,
    'product_category_name_translation_dataset': product_category_name_translation_dataset
}

# using a for loop, run the different functions to check the necessary

In [10]:
for key,values in dataframes.items():
    check_data_types(values)
    print(" ")
    check_data_shape(values)
    check_unique_rows(values)
    print(" ")
    check_null_values(values)
    print(" ")
    print(" ")

customers_dataset has the following data types:
customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object
 
The customers_dataset has the following shape, 99441 rows & 5 columns.
The customers_dataset has 99441 unique rows.
 
The customers_dataset has the following missing values:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64
 
 
geolocation_dataset has the following data types:
geolocation_zip_code_prefix      int64
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
dtype: object
 
The geolocation_dataset has the following shape, 1000163 rows & 5 columns.
The geolocation_dataset has 738332 unique rows.
 
The geolocation_dataset has the fo

# customers_dataset cleaning
#### 1. Ensure `customer_zip_code_prefix` is numeric and standardize its length if needed

In [11]:
# Ensure `customer_zip_code_prefix` is numeric and standardize its length if needed
customers_dataset['customer_zip_code_prefix'] = customers_dataset['customer_zip_code_prefix'].astype(str).str.zfill(5)
#customers_dataset

# geo_dataset cleaning:

#### 1. standardizing geolocation_zip_code to 5 characters

In [12]:
# Ensure `geolocation_zip_code_prefix` is numeric and standardize its length if needed
geolocation_dataset['geolocation_zip_code_prefix'] = geolocation_dataset['geolocation_zip_code_prefix'].astype(str).str.zfill(5)
#geo_dataset

#### 2. standardizing the city, state names to lowercase & removing whitespaces

In [13]:
# Standardizing the city and state names by converting them to lowercase and stripping any leading/trailing whitespace
geolocation_dataset['geolocation_city'] = geolocation_dataset['geolocation_city'].str.lower().str.strip()
geolocation_dataset['geolocation_state'] = geolocation_dataset['geolocation_state'].str.upper().str.strip()

#geo_dataset

#### 3. creating a mapping dictionary for known city name variations

In [14]:
# Correcting common city name variations
# Creating a mapping dictionary for known variations
city_corrections = {
    'sao paulo': 'sao paulo',
    'sãopaulo': 'sao paulo',
    'são paulo': 'sao paulo',
    'sa£o paulo': 'sao paulo',
    'sp': 'sao paulo',  # assuming 'sp' is intended to be 'são paulo'
    'carapicuiba': 'carapicuíba',
    'sao bernardo do campo': 'sao bernardo do campo',
    'taboão da serra': 'taboao da serra',
    'jundiaí': 'jundiaí',
    # Add more corrections as needed
}

# Applying the corrections
geolocation_dataset['geolocation_city'] = geolocation_dataset['geolocation_city'].replace(city_corrections)
#geo_dataset

# order_items_dataset datetime conversion

#### 1. Convert 'shipping_limit_date' to datetime format

In [15]:
# Convert 'shipping_limit_date' to datetime format
order_items_dataset['shipping_limit_date'] = pd.to_datetime(order_items_dataset['shipping_limit_date'])
#order_items_dataset

# order_reviews_dataset
#### 1. Fill null values with a constant value in order_reviews_dataset

In [16]:
print(order_reviews_dataset.isna().sum())

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64


In [17]:
#Fill null values with a constant value in order_reviews tabe
order_reviews_dataset["review_comment_title"] = order_reviews_dataset["review_comment_title"].fillna("No Title")
order_reviews_dataset["review_comment_message"] = order_reviews_dataset["review_comment_message"].fillna("No Comment")

In [18]:
print(order_reviews_dataset.isna().sum())

review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
dtype: int64


#### 2. Remove rows with any null values in orders table

In [19]:
#check for null values in orders_dataset
orders_dataset.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [20]:
change_to_datetime = {'order_purchase_timestamp':'order_purchase_timestamp',
                      'order_approved_at':'order_approved_at',
                      'order_delivered_carrier_date':'order_delivered_carrier_date',
                      'order_delivered_customer_date':'order_delivered_customer_date',
                      'order_estimated_delivery_date':'order_estimated_delivery_date'}

def change_all_columns_to_datetime():
    for i in change_to_datetime:
        #print(i)
        orders_dataset[i] = pd.to_datetime(orders_dataset[i])
        return orders_dataset.info()

change_all_columns_to_datetime()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  object        
 5   order_delivered_carrier_date   97658 non-null  object        
 6   order_delivered_customer_date  96476 non-null  object        
 7   order_estimated_delivery_date  99441 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 6.1+ MB


In [21]:
orders_dataset.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [22]:
def fill_timestamp():
    for x in orders_dataset:
        orders_dataset[x] = orders_dataset[x].fillna(pd.Timestamp("1970-01-01 00:00:00"))
    return orders_dataset.isna().sum()

fill_timestamp()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
dtype: int64

# products_dataset

#### 1. rename column names

In [23]:
#rename column names
products_dataset = products_dataset.rename(columns={'product_name_lenght':'product_name_length', 
                                                    'product_description_lenght':'product_description_length', 
                                                    'product_weight_g':'product_weight_grams'})

 #### 2. Replacing product_category_name with translated version & corrected spelling in values.

In [24]:
# Create a dictionary from the translation dataset
translation_dict = dict(zip(
    product_category_name_translation_dataset['product_category_name'],
    product_category_name_translation_dataset['product_category_name_english']))

# Use map to translate product category names
products_dataset['product_category_name'] = products_dataset['product_category_name'].map(translation_dict)

In [25]:
products_dataset['product_category_name'] = products_dataset['product_category_name'].replace({'auto':'automotives', 'perfumery':'perfume', 
                                                                                       'telephony':'handphone', 'baby':'baby_items', 'stationery':'stationaries',
                                                                                       'fixed_telephony': 'telephone', 'home_confort':'house_comfort'})

#### 3. Check for null values in each row

In [26]:
products_dataset.isnull().sum()

product_id                      0
product_category_name         623
product_name_length           610
product_description_length    610
product_photos_qty            610
product_weight_grams            2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

#### 4. fill empty values with appropriate values to preserve the row.

In [27]:
def products_fill_values():
    products_dataset['product_category_name'] = products_dataset['product_category_name'].fillna('No Name')
    products_dataset['product_name_length'] = products_dataset['product_name_length'].fillna(0)
    products_dataset['product_length_cm'] = products_dataset['product_length_cm'].fillna(0)
    products_dataset['product_description_length'] = products_dataset['product_description_length'].fillna(0)
    products_dataset['product_photos_qty'] = products_dataset['product_photos_qty'].fillna(0)
    products_dataset['product_weight_grams'] = products_dataset['product_weight_grams'].fillna(0)
    products_dataset['product_length_cm'] = products_dataset['product_length_cm'].fillna(0)
    products_dataset['product_height_cm'] = products_dataset['product_height_cm'].fillna(0)
    products_dataset['product_width_cm'] = products_dataset['product_width_cm'].fillna(0)
    return products_dataset.isnull().sum()

In [28]:
#final check for missing values in products dataset
products_fill_values()

product_id                    0
product_category_name         0
product_name_length           0
product_description_length    0
product_photos_qty            0
product_weight_grams          0
product_length_cm             0
product_height_cm             0
product_width_cm              0
dtype: int64

# sellers_dataset cleaning

#### 1. change zip_code_prefix to 5 characters to match with geolocation_zip_code_prefix

In [29]:
sellers_dataset['seller_zip_code_prefix'] = sellers_dataset['seller_zip_code_prefix'].astype(str).str.zfill(5)

In [30]:
sellers_dataset['seller_zip_code_prefix']

0       13023
1       13844
2       20031
3       04195
4       12914
        ...  
3090    87111
3091    88137
3092    04650
3093    96080
3094    12051
Name: seller_zip_code_prefix, Length: 3095, dtype: object

In [31]:
orders_dataset.to_csv('orders_dataset.csv', index=False)
order_items_dataset.to_csv('order_items_dataset.csv', index=False)
order_payments_dataset.to_csv('order_payments_dataset.csv', index=False)
order_reviews_dataset.to_csv('order_reviews_dataset.csv', index=False)
customers_dataset.to_csv('customers_dataset.csv', index=False)
products_dataset.to_csv('products_dataset.csv', index=False)
sellers_dataset.to_csv('sellers_dataset.csv', index=False)
geolocation_dataset.to_csv('geolocation_dataset.csv', index=False)

# Tables Creation

**Connecting to postgres database**

In [32]:
try: 
    conn = psycopg2.connect(
        host="localhost",
        port="5432",  # Specify port as a separate argument
        dbname="postgres",
        user="postgres",
        password="password"  # Replace 'your_password' with the actual password
    )
    print("Connection successful")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)


Connection successful


In [33]:
#Use connection to get cursor that can be used to execute queries.
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the Database")
    print(e)

In [34]:
#Set automatic commit to be true so that each action is committed without having to call conn.commit() after each command
conn.set_session(autocommit=True)

In [35]:
#Create database to do work in
try:
    cur.execute("create database final_project")
except psycopg2.Error as e:
    print(e)

database "final_project" already exists



**Adding final DB**

In [36]:
try:
    conn.close()
except psycopg2.Error as e:
    print(e)

try:
    conn = psycopg2.connect(host="localhost",
        port="5432",  # Specify port as a separate argument
        dbname="final_project",
        user="postgres",
        password="password"  # Replace 'your_password' with the actual password
        )
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the Database")
    print(e)

conn.set_session(autocommit=True)

# Creating_Tables
#### orders, order_reviews, order_payments, order_reviews, customers

In [37]:
try:
    cur.execute("""
    CREATE TABLE IF NOT EXISTS orders (
        order_id TEXT UNIQUE NOT NULL,
        customer_id TEXT UNIQUE NOT NULL,
        order_status TEXT,
        order_purchase_timestamp TIMESTAMP,
        order_approved_at TIMESTAMP,
        order_delivered_carrier_date TIMESTAMP,        
        order_delivered_customer_date TIMESTAMP,
        order_estimated_delivery_date TIMESTAMP
    ) 
    """)
  
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [38]:
try:
    cur.execute("""
    CREATE TABLE IF NOT EXISTS order_reviews (
        review_id TEXT ,
        review_score NUMERIC,
        review_comment_title TEXT,      
        review_comment_message TEXT,
        review_creation_date TIMESTAMP,
        review_answer_timestamp TIMESTAMP,
        order_id TEXT UNIQUE NOT NULL,
        PRIMARY KEY (order_id)
    )
    """)
   
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [39]:
try:
    cur.execute("""
    CREATE TABLE IF NOT EXISTS order_payments (
        order_id TEXT,
        payment_sequential NUMERIC,
        payment_type TEXT,
        payment_installments NUMERIC,
        payment_value DECIMAL(10, 2)
    )
    """)
    
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [40]:
#order_items
try: 
    cur.execute("""
    CREATE TABLE IF NOT EXISTS order_items (
        order_id TEXT,
        order_item_id TEXT,
        product_id TEXT UNIQUE NOT NULL,
        seller_id TEXT UNIQUE NOT NULL,
        shipping_limit_date TIMESTAMP,
        price DOUBLE PRECISION,
        freight_value DOUBLE PRECISION
        );
    """)
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print(e)

In [41]:
#customer table
try: 
    cur.execute("""
    CREATE TABLE IF NOT EXISTS customers (
        customer_id TEXT UNIQUE NOT NULL,
        customer_unique_id TEXT,
        customer_zip_code_prefix INT UNIQUE NOT NULL,
        customer_city TEXT,
        customer_state TEXT,
        PRIMARY KEY (customer_id)    
    );
    """)
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print(e)

# Adding in constraints & foreign keys to respective tables IN ORDER
#### order_reviews, order_payments, customers, order_items

In [42]:
#adding constraint FK to order_reviews REFERENCING orders
try:
    cur.execute("""
    ALTER TABLE order_reviews
        ADD CONSTRAINT order_reviews_fkey
            FOREIGN KEY (order_id)
                REFERENCES orders (order_id)
        """)

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

Error: Issue creating table
constraint "order_reviews_fkey" for relation "order_reviews" already exists



In [43]:
#adding constraint FK to order_payments REFERENCING orders
try:
    cur.execute("""
    ALTER TABLE order_payments
        ADD CONSTRAINT order_payments_fkey
            FOREIGN KEY (order_id)
                REFERENCES orders (order_id)
        """)

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

Error: Issue creating table
constraint "order_payments_fkey" for relation "order_payments" already exists



In [44]:
#adding constraint FK to customers REFERENCING orders
try:
    cur.execute("""
    ALTER TABLE customers
        ADD CONSTRAINT customers_fkey
            FOREIGN KEY (customer_id)
                REFERENCES orders (customer_id)
        """)

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

Error: Issue creating table
constraint "customers_fkey" for relation "customers" already exists



In [45]:
#adding constraint FK to order_items REFERENCING orders
try:
    cur.execute("""
    ALTER TABLE order_items
        ADD CONSTRAINT order_items_fkey
            FOREIGN KEY (order_id)
                REFERENCES orders (order_id)
        """)

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

Error: Issue creating table
constraint "order_items_fkey" for relation "order_items" already exists



# creating Products table & adding in constraints & foreign key

In [46]:
try:
    cur.execute("""
    CREATE TABLE IF NOT EXISTS products (
        product_id TEXT UNIQUE NOT NULL,
        product_name_length NUMERIC,
        product_description_length NUMERIC,
        product_photos_qty NUMERIC,
        product_weight_grams NUMERIC,
        product_length_cm NUMERIC,
        product_height_cm NUMERIC,
        product_width_cm NUMERIC,
        product_category_name TEXT,
        PRIMARY KEY (product_id)
    )
    """)
    
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [47]:
#adding constraint FK to products REFERENCING order_items
try:
    cur.execute("""
    ALTER TABLE products
        ADD CONSTRAINT products_fkey
            FOREIGN KEY (product_id)
                REFERENCES order_items (product_id)
        """)

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

Error: Issue creating table
there is no unique constraint matching given keys for referenced table "order_items"



# creating sellers table & adding in constraints & foreign key

In [48]:
try:
    cur.execute("""
    CREATE TABLE IF NOT EXISTS sellers (
        seller_id TEXT UNIQUE NOT NULL,
        seller_zip_code_prefix INT UNIQUE NOT NULL,
        seller_city TEXT,
        seller_state TEXT,
        PRIMARY KEY (seller_id)
    )
    """)
    
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [49]:
#adding constraint FK to sellers REFERENCING order_items
try:
    cur.execute("""
    ALTER TABLE sellers
        ADD CONSTRAINT seller_id_fkey
            FOREIGN KEY (seller_id)
                REFERENCES order_items (seller_id)
        """)

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

Error: Issue creating table
there is no unique constraint matching given keys for referenced table "order_items"



# creating geolocation table & adding in constraints & 2 foreign keys (sellers & customers)

In [50]:
#geolocation table
try: 
    cur.execute("""
    CREATE TABLE IF NOT EXISTS geolocation (
        geolocation_zip_code_prefix INT UNIQUE NOT NULL,
        geolocation_lat DOUBLE PRECISION,
        geolocation_lng DOUBLE PRECISION,
        geolocation_city TEXT,
        geolocation_state TEXT,
        PRIMARY KEY (geolocation_zip_code_prefix)
        
    );
    """)
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print(e)

In [51]:
#adding constraint FK to geolocation REFERENCING sellers
try:
    cur.execute("""
    ALTER TABLE geolocation
    ADD CONSTRAINT geolocation_zip_code_prefix_fkey
        FOREIGN KEY (geolocation_zip_code_prefix)
            REFERENCES sellers (seller_zip_code_prefix)
        """)

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

Error: Issue creating table
there is no unique constraint matching given keys for referenced table "sellers"



In [52]:
#adding constraint FK to geolocation REFERENCING customers
try:
    cur.execute("""
    ALTER TABLE geolocation
    ADD CONSTRAINT geolocation_zip_code_prefix_fkey1
        FOREIGN KEY (geolocation_zip_code_prefix)
            REFERENCES customers (customer_zip_code_prefix)
        """)

except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

Error: Issue creating table
there is no unique constraint matching given keys for referenced table "customers"



# Inserting records to tables in reverse order of creation
#### geolocation, sellers, products, customers, order_items, order_payments, order_reviews, orders

In [53]:
# PostgreSQL connection details
username = "postgres"
password = "password"
host = "localhost"
port = "5432"  # Specify port as a separate argument
database = "final_project"

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

In [54]:
geolocation_dataset.to_sql('geolocation', con=engine, if_exists='replace', index=False)
# Print success message
print("Data successfully written to the 'customers' table.")

Data successfully written to the 'customers' table.


In [55]:
sellers_dataset.to_sql('sellers', con=engine, if_exists='replace', index=False)
# Print success message
print("Data successfully written to the 'customers' table.")

Data successfully written to the 'customers' table.


In [56]:
products_dataset.to_sql('products', con=engine, if_exists='replace', index=False)
#Print success message
print("Data successfully written to the 'customers' table.")

Data successfully written to the 'customers' table.


In [57]:
customers_dataset.to_sql('customers', con=engine, if_exists='replace', index=False)
# Print success message
print("Data successfully written to the 'customers' table.")

Data successfully written to the 'customers' table.


In [58]:
order_items_dataset.to_sql('order_items', con=engine, if_exists='replace', index=False)
# Print success message
print("Data successfully written to the 'customers' table.")

Data successfully written to the 'customers' table.


In [59]:
order_payments_dataset.to_sql('order_payments', con=engine, if_exists='replace', index=False)
# Print success message
print("Data successfully written to the 'customers' table.")

Data successfully written to the 'customers' table.


In [60]:
order_reviews_dataset.to_sql('order_reviews', con=engine, if_exists='replace', index=False)
# Print success message
print("Data successfully written to the 'customers' table.")

Data successfully written to the 'customers' table.


In [61]:
# Write the DataFrame to PostgreSQL
orders_dataset.to_sql('orders', con=engine, if_exists='replace', index=False)
# Print success message
print("Data successfully written to the 'customers' table.")

Data successfully written to the 'customers' table.
